### 1. 先简单介绍一下
（不知道如何介绍于是直接翻译官网了-v-）
<br>tf.data这个模块包含一系列Classes，允许你轻易地加载数据集，操作它，然后将它传入到你的模型中.
<br>这个tutorial呢，从两个简单的例子来介绍这个API:
* 通过Numpy提供的array，从内存中读入数据
* 从csv文件中读入行

### 2.超基本的输入
从数组中读取是最简单的方式，所以我们先从最简单的开始好啦

In [12]:
import tensorflow as tf
import iris_data

train, test = tf.keras.datasets.mnist.load_data()
mnist_x, mnist_y = train

mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)

### 2. tf.data.Dataset.from_tensor_slices() 中的slice是什么概念？
首先呢，这个函数需要数组(numpy.array)作为输入，并返回一片一片(slice)的array，嗯，这个一片一片的意思就是，拿上面的Mnist数据集来说，输入的是大小为（60000，28，28）的数组，返回的是60000片，每片大小为28*28，这样的数据集
<br> 我们来打印一下看看 👇

In [13]:
print(mnist_ds)

<TensorSliceDataset shapes: (28, 28), types: tf.uint8>


打印出的信息显示了这一片的大小以及数据类型，但需要注意的是，这一片并不知道它有多少小伙伴，意思就是，我们打印的这一片，不知道一共有多少篇

### 3. tf.data.Dataset.from_tensor_slices()中的其他参数呢？
array类型的就好，但其实几乎任何数据类型都可以通过numpy.array被转换为array类型，例如下面这个例子：

In [22]:
train, test = iris_data.load_data()
features, labels = train
dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

In [23]:
print(dataset)

<TensorSliceDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


像这种结构化(dict,array)元素，返回的Dataset的大小（shape）和类型（type）采取同样的结构

### 4.然后就可以用了吗？
不，还需要进一步处理，目前返回的Dataset每次迭代返回一片数据且以一个固定的顺序，所以，可以进一步操作的有：
* Shuffle 改变顺序用的，有一个参数是buffer_size,确保提供大于数据集的空间，以便彻底打乱顺序
* Repeat 字面意思
* Batch 一次传入一叠数据（就是传入很多片的意思） 其实batch是一个矩阵化，涉及到效率问题，虽然很重要，但是写起来太麻烦了，但还是推荐看一下。

In [18]:
# 显然是没有用且没效果的打乱
print(mnist_ds.shuffle(100)) 

<ShuffleDataset shapes: (28, 28), types: tf.uint8>


In [19]:
#一次传入一百片 -v-
print(mnist_ds.batch(100))

<BatchDataset shapes: (?, 28, 28), types: tf.uint8>


不过你大概注意到，输出的大小是(?,28,28),而不是(100,28,28),是因为，最后一次传入数据时，也许没办法凑整，会小一点，所以这个大小没办法确定，就用？来代替了

### 5.但是在训练的过程中，也需要特征和标签，该如何加入返回的数据集呢？
就是，例如从颜色大小重量来判断是不是苹果，那么模型需要知道突然传入的四个数值，谁是颜色大小重量和标签...
<br>dataset提供以下这种方式将数据转化为需要的格式

In [24]:
# Build the Iterator, and return the read end of the pipeline.
features_result, labels_result = dataset.make_one_shot_iterator().get_next()

In [25]:
print((features_result, labels_result))

({'SepalLength': <tf.Tensor 'IteratorGetNext_1:2' shape=() dtype=float64>, 'SepalWidth': <tf.Tensor 'IteratorGetNext_1:3' shape=() dtype=float64>, 'PetalLength': <tf.Tensor 'IteratorGetNext_1:0' shape=() dtype=float64>, 'PetalWidth': <tf.Tensor 'IteratorGetNext_1:1' shape=() dtype=float64>}, <tf.Tensor 'IteratorGetNext_1:4' shape=() dtype=int64>)


### 6.另一种读入数据方式 ---- 从CSV文件中读入数据

In [30]:
train_path, test_path = iris_data.maybe_download()

#### 建立数据集
首先我们建立一个TextLineDataset对象，每次从文件中读入一行数据。
<br>然后，我们调用skip方法，来跳过文件的第一行（因为只是个标题，不是数据）

In [27]:
ds = tf.data.TextLineDataset(train_path).skip(1)

#### 建立csv分割元
我们需要将每一行分割为（特征，标签）对
<br>所以，我们需要根据自己的需求，建立分割元
<br>官网上给了样例

In [34]:
# Metadata describing the text columns
COLUMNS = ['SepalLength', 'SepalWidth',
           'PetalLength', 'PetalWidth',
           'label']
FIELD_DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [0]]
def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, FIELD_DEFAULTS)

    # Pack the result into a dictionary
    features = dict(zip(COLUMNS,fields))

    # Separate the label from the features
    labels= features.pop('label')

    return features, labels

* <a href = "https://www.tensorflow.org/api_docs/python/tf/decode_csv?hl=zh-cn">tf.decode_csv()</a>: 将CSV记录转化为tensors <br>
* zip()：是python提供的打包函数，例如a = [1,2,3],b = [4,5,6],那么返回的zip(a,b)就是[(1,4),(2,5),(3,6)],嗯也就是，首先打包成元组，再返回由元组组成的列表
* pop():队列中的pop(),可以理解为出队，从features中找到名为“label”的货，让它转队到labels

In [35]:
ds = ds.map(_parse_line)
print(ds)

<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>


* map()是Dataset自带的函数，需要传入的参数是自定义的映射函数，例如我们刚刚定义的_parse_line()

以上，这一部分的tutorial结束啦=w=